<a href="https://colab.research.google.com/github/YUTA-NIKI/learnin_dl-/blob/master/chap03_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture3 Homework
Make  softmax model to clasify Fashion MNIST (https://github.com/zalandoresearch/fashion-mnist).

## Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [13]:
!ls /content/drive/"My Drive"/lecture_master/2019_summer/deeplearning_matsuo/chap03_data

x_test.npy  x_train.npy  y_train.npy


## Read Data

In [0]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('/content/drive/My Drive/lecture_master/2019_summer/deeplearning_matsuo/chap03_data/x_train.npy')
    y_train = np.load('/content/drive/My Drive/lecture_master/2019_summer/deeplearning_matsuo/chap03_data/y_train.npy')
    
    # テストデータ
    x_test = np.load('/content/drive/My Drive/lecture_master/2019_summer/deeplearning_matsuo/chap03_data/x_test.npy')
    
    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

## Implement Softmax Regression

In [20]:
# %%writefile /root/userspace/chap03/submission_code.py

x_train, y_train, x_test = load_fashionmnist()

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def softmax(x):
    x -= x.max(axis=1, keepdims=True) # expのunderflow & overflowを防ぐ
    x_exp = np.exp(x)
    return x_exp / np.sum(x_exp, axis=1, keepdims=True)

def np_log(x):
    return np.log(np.clip(a=x, a_min=1e-10, a_max=x))

# weights
W = np.random.uniform(low=-0.08, high=0.08, size=(784, 10)).astype('float32')
b = np.zeros(shape=(10,)).astype('float32')

# 学習データと検証データに分割
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1)

def train(x, t, eps=1.0):
    
    global W, b
    
    batch_size = x.shape[0]
    
    y = softmax(np.matmul(x, W) + b)
    
    cost = (- t * np_log(y)).sum(axis=1).mean()
    delta = y - t
    
    dW = np.matmul(x.T, delta) / batch_size
    db = np.matmul(np.ones(shape=(batch_size,)), delta) / batch_size
    
    W -= eps * dW
    b -= eps * db
    
    return cost

def valid(x, t):
    y = softmax(np.matmul(x, W) + b)
    cost = (-t * np_log(y)).sum(axis=1).mean()
    
    return cost, y

for epoch in range(3):
    # オンライン学習
    x_train, y_train = shuffle(x_train, y_train)
    for x, y in zip(x_train, y_train):
        cost = train(x[None, :], y[None, :])
    cost, y_pred = valid(x_valid, y_valid)
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
        epoch + 1,
        cost,
        accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
    ))

y_pred = softmax(np.matmul(x_test, W) + b)

submission = pd.Series(y_pred.argmax(axis=1), name='label')
submission.to_csv('/content/drive/My Drive/lecture_master/2019_summer/deeplearning_matsuo/homework/chap03/submission_pred.csv', header=True, index_label='id')

EPOCH: 1, Valid Cost: 4.782, Valid Accuracy: 0.780
EPOCH: 2, Valid Cost: 7.355, Valid Accuracy: 0.670
EPOCH: 3, Valid Cost: 4.661, Valid Accuracy: 0.784


## Grid Search

In [31]:
x_train, y_train, x_test = load_fashionmnist()

# 学習データと検証データに分割
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1)

eps_params   = [0.01, 0.1, 1]
epoch_params = [1,3,5,10]

best_score = 0
best_param = {}

for eps in eps_params:
    for epoch in epoch_params:
        
        # weights
        W = np.random.uniform(low=-0.08, high=0.08, size=(784, 10)).astype('float32')
        b = np.zeros(shape=(10,)).astype('float32')
        
        for epo_num in range(epoch):
            # オンライン学習
            x_train, y_train = shuffle(x_train, y_train)
            for x, y in zip(x_train, y_train):
                cost = train(x[None, :], y[None, :], eps = eps)
            cost, y_pred = valid(x_valid, y_valid)
            print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
                epo_num + 1,
                cost,
                accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
            ))
            
        y_pred = softmax(np.matmul(x_valid, W) + b)
        score = accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
        
        if score > best_score:
            best_score = score
            best_param = {'eps': eps, 'epoch': epoch}
        print('-----------------------------------------------------------------')
        
print('best_score: ', best_score)
print('best_param; ', best_param)

# weights
W = np.random.uniform(low=-0.08, high=0.08, size=(784, 10)).astype('float32')
b = np.zeros(shape=(10,)).astype('float32')

for epoch in range(best_param['epoch']):
    # オンライン学習
    x_train, y_train = shuffle(x_train, y_train)
    for x, y in zip(x_train, y_train):
        cost = train(x[None, :], y[None, :], eps=best_param['eps'])
    cost, y_pred = valid(x_valid, y_valid)
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
        epoch + 1,
        cost,
        accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
    ))
    
y_pred = softmax(np.matmul(x_test, W) + b)

submission = pd.Series(y_pred.argmax(axis=1), name='label')
submission.to_csv('/content/drive/My Drive/lecture_master/2019_summer/deeplearning_matsuo/homework/chap03/submission_pred.csv', header=True, index_label='id')

EPOCH: 1, Valid Cost: 0.488, Valid Accuracy: 0.844
-----------------------------------------------------------------
EPOCH: 1, Valid Cost: 0.566, Valid Accuracy: 0.815
EPOCH: 2, Valid Cost: 0.514, Valid Accuracy: 0.832
EPOCH: 3, Valid Cost: 0.614, Valid Accuracy: 0.807
-----------------------------------------------------------------
EPOCH: 1, Valid Cost: 0.521, Valid Accuracy: 0.823
EPOCH: 2, Valid Cost: 0.482, Valid Accuracy: 0.845
EPOCH: 3, Valid Cost: 0.563, Valid Accuracy: 0.824
EPOCH: 4, Valid Cost: 0.594, Valid Accuracy: 0.814
EPOCH: 5, Valid Cost: 0.539, Valid Accuracy: 0.835
-----------------------------------------------------------------
EPOCH: 1, Valid Cost: 0.486, Valid Accuracy: 0.836
EPOCH: 2, Valid Cost: 0.466, Valid Accuracy: 0.847
EPOCH: 3, Valid Cost: 0.529, Valid Accuracy: 0.836
EPOCH: 4, Valid Cost: 0.462, Valid Accuracy: 0.847
EPOCH: 5, Valid Cost: 0.479, Valid Accuracy: 0.843
EPOCH: 6, Valid Cost: 0.517, Valid Accuracy: 0.829
EPOCH: 7, Valid Cost: 0.552, Valid Ac